    # Automata and Computability Homework 2 problem 1
Repo: https://github.com/TheCDC/CSC413

This file: https://github.com/TheCDC/CSC413/blob/master/HW2/HW2-1.ipynb

# Implement the given NFA

In [9]:
def delta_nfa(state, char):
    """Delta function for NFA in problem."""
    d = {
        ('s','a'): {'s','t'},
        ('s','b'): {'s',},
        ('t','a'): {'u','s'},
        ('u','a'): {'v','s'},
    }
    
    return d.get((state,char), set())

Test by eye

In [10]:
delta_nfa('s','a')

{'s', 't'}

In [11]:
all_states = set('stuv')

Create helper functions

In [12]:
from itertools import chain, combinations
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

Construct all subsets of states

In [13]:
alphabet = set('ab')
all_subsets = [frozenset(i) for i in powerset(all_states)]
all_subsets

[frozenset(),
 frozenset({'t'}),
 frozenset({'s'}),
 frozenset({'v'}),
 frozenset({'u'}),
 frozenset({'s', 't'}),
 frozenset({'t', 'v'}),
 frozenset({'t', 'u'}),
 frozenset({'s', 'v'}),
 frozenset({'s', 'u'}),
 frozenset({'u', 'v'}),
 frozenset({'s', 't', 'v'}),
 frozenset({'s', 't', 'u'}),
 frozenset({'t', 'u', 'v'}),
 frozenset({'s', 'u', 'v'}),
 frozenset({'s', 't', 'u', 'v'})]

Construct all possible transitions and print


In [14]:
import pandas as pd
intermediate_mapping = dict()
# loop over subsets
for subset in all_subsets:
#     loop over alphabet
    for char in alphabet:    
        next_state = set()
#         find destinations from all states
        for state in subset:
            ss = delta_nfa(state, char)
            next_state.update(ss)
            intermediate_mapping.update({(subset,char):frozenset(next_state)})

# hack to make formatting nice
things = [{'source':set(i[0][0]),'character':i[0][1],'destination':set(i[1])} for i in intermediate_mapping.items()]
df = pd.DataFrame(things)
df[['source','character','destination']]

,source,character,destination
0,{u},b,{}
1,"{t, s}",b,{s}
2,"{t, s}",a,"{u, s, t}"
3,{s},b,{s}
4,"{u, v}",a,"{s, v}"
5,{s},a,"{t, s}"
6,"{s, v}",a,"{t, s}"
7,"{u, s}",b,{s}
8,"{t, s, u, v}",a,"{u, s, t, v}"
9,"{t, u, v}",b,{}


# Prune states that can never be reached

Some sets of states form 'islands' that can never be reached from any start state. These are superfluous and can be pruned to produce a more sensible DFA.

In [15]:
# perform graph traversal to prune inaccessible states
start_state = 's'
# remember which states have been traversed
seen_states = set()
# perform traversal iteratively using a stack
states_stack = [{'s'}]
while len(states_stack) > 0:
#     pop off a node
    state = frozenset(states_stack.pop())
    seen_states.add(state)
    print('saw',state)
#     construct the set of children of this node using the delta function
    for char in alphabet:
        ns = set()
        for substate in state:
            child = delta_nfa(substate, char)
            ns.update(child)
        if ns not in seen_states:
            states_stack.append(ns)
# delete all ((state,char)=> states) pairs from the mapping
# where the state was not traversed
print("Deleting all other states...")
pruned_mapping=dict(intermediate_mapping)
for key in list(pruned_mapping.keys()):
    if key[0] not in seen_states :
#         print('delete inaccessible state',set(key),'=>',set(pruned_mapping[key]))
        del pruned_mapping[key]

saw frozenset({'s'})
saw frozenset({'t', 's'})
saw frozenset({'u', 's', 't'})
saw frozenset({'t', 's', 'u', 'v'})
Deleting all other states...


In [16]:
things = [{'source':set(i[0][0]),'character':i[0][1],'destination':set(i[1])} for i in pruned_mapping.items()]
df = pd.DataFrame(things)
df[['source','character','destination']]

,source,character,destination
0,"{t, s}",b,{s}
1,"{t, s}",a,"{u, s, t}"
2,{s},b,{s}
3,{s},a,"{t, s}"
4,"{t, s, u, v}",a,"{u, s, t, v}"
5,"{t, s, u}",a,"{u, s, t, v}"
6,"{t, s, u}",b,{s}
7,"{t, s, u, v}",b,{s}
